In [1]:
import glob
#긍정리뷰 100개 불러오기
pos_review=(glob.glob("c:/data/imdb/train/pos/*.txt"))[0:100]
lines_pos=[]
for i in pos_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except :
        continue
len(lines_pos)

100

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
tokenizer = RegexpTokenizer('[\w]+')
stop_words = stopwords.words('english')
#TF-IDF 가중치 할당
vec = TfidfVectorizer(stop_words=stop_words)
vector_lines_pos = vec.fit_transform(lines_pos)
A=vector_lines_pos.toarray()
print(A.shape)
print(A)
#x축 단어, y축 문서

(100, 3989)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.06538462 0.         ... 0.         0.         0.        ]
 [0.         0.23078109 0.         ... 0.         0.         0.        ]]


In [3]:
#현재 상태는 100개의 문서의 유사도
#단어간의 유사도를 구하는 것이 목적이므로
#단어-문서 행렬로 변경
#x축 문서, y축 단어
A=A.transpose()
print(A.shape)
print(A)

(3989, 100)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.06538462 0.23078109]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [4]:
import numpy as np
from scipy import sparse
#밀집행렬(dense array)
a=np.array([[0.5,0,0],[0,1,0],[0.7,0,1.5]])
#밀집행렬을 희소행렬(sparse array)로 변환
#밀집행렬의 단점: 0이 많을 경우 메모리 낭비가 될 수 있음
#희소행렬은 0이 아닌 값들의 위치와 값만 기록하여 메모리를 절약하는 방식
b=sparse.csr_matrix(a)
print(b)
# (0,0) 0.5 => 인덱스 0,0에 값 0.5
# (1,1) 1 => 인덱스 1,1에 값 1
# (2,0) 0.7 => 인덱스 2,0에 값 0.7
# (2,2) 1 => 인덱스 2,2에 값 1.5
c=b.toarray() #희소행렬을 밀집행렬로 변환
print(c)

  (0, 0)	0.5
  (1, 1)	1.0
  (2, 0)	0.7
  (2, 2)	1.5
[[0.5 0.  0. ]
 [0.  1.  0. ]
 [0.7 0.  1.5]]


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
A_sparse = sparse.csr_matrix(A)
#코사인 유사도 계산
similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
# todok() 행렬을 딕셔너리 형태로 변환
list(similarities_sparse.todok().items())[35000:35010]
#list(similarities_sparse.todok().items())[-10:]

[((1556, 108), 0.1025903571594213),
 ((1557, 108), 0.1397555705460018),
 ((1563, 108), 0.304841238021036),
 ((1574, 108), 0.304841238021036),
 ((1575, 108), 0.1373985754191761),
 ((1588, 108), 0.08716652865566843),
 ((1589, 108), 0.304841238021036),
 ((1598, 108), 0.12432181608698951),
 ((1606, 108), 0.1835286370686118),
 ((1607, 108), 0.304841238021036)]

In [8]:
#단어 이름이 아닌 인덱스 형태로 출력됨
# (1469,108), 0.37 1469 단어와 108 단어의 유사도는 37%
vec.get_feature_names_out()[1469]

'friday'

In [6]:
import pandas as pd
import numpy as np
#위의 결과값을 데이터프레임으로 출력
df=pd.DataFrame(list(similarities_sparse.todok().items()),columns=["words","weight"])
df2=df.sort_values(by=['weight'],ascending=False)
df2=df2.reset_index(drop=True)
#단어 자신끼리의 짝은 1이 되므로 1보다 작은 항목들만 출력시킴
df3=df2.loc[np.round(df2['weight']) < 1]
df3=df3.reset_index(drop=True)
df3.head(10)

,words,weight
0,"(2613, 3250)",0.499980
1,"(3250, 2613)",0.499980
2,"(2545, 1188)",0.499979
3,"(1188, 2545)",0.499979
4,"(1767, 2543)",0.499965
5,"(2543, 1767)",0.499965
6,"(1074, 250)",0.499935
7,"(250, 1074)",0.499935
8,"(2322, 525)",0.499934
9,"(525, 2322)",0.499934


In [9]:
for i,row in enumerate(df3.iterrows()):
    a=vec.get_feature_names_out()[row[1][0][0]]
    b=vec.get_feature_names_out()[row[1][0][1]]
    print(f'{a},{b}=>{row[1][1]:.2f}')    
    if i>10:
        break

physically,society=>0.50
society,physically=>0.50
past,essence=>0.50
essence,past=>0.50
iceberg,passengers=>0.50
passengers,iceberg=>0.50
dying,art=>0.50
art,dying=>0.50
move,care=>0.50
care,move=>0.50
horner,long=>0.50
long,horner=>0.50
